In [0]:
!pip install wordbatch

In [0]:
!pip install --upgrade pandas
!pip install --upgrade numpy

In [0]:
!pip install catboost

In [0]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

import seaborn as sns
sns.set_style('whitegrid')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from sklearn.pipeline import make_union
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack
from scipy.sparse import vstack
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize
from sklearn.feature_selection.univariate_selection import SelectKBest, f_regression

import wordbatch
from wordbatch.models import FTRL, FM_FTRL
from sklearn.linear_model import Ridge
from wordbatch.extractors import WordBag, WordHash

from sklearn.naive_bayes import MultinomialNB
from catboost import CatBoostRegressor

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
all_df = pickle.load(open('/content/gdrive/My Drive/MerCari/all_df.p', 'rb'))
all_df.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,general_category,sub_cat_1,sub_cat_2
0,0,mlb cincinnati red shirt size,3,None,2.397895,1,descript yet,Men,Tops,T-shirts
1,1,razer blackwidow chroma keyboard,3,Razer,3.970292,0,keyboard great condit work like came box port ...,Electronics,Computers & Tablets,Components & Parts
2,2,ava viv blous,1,Target,2.397895,1,ador top hint lace key hole back pale pink als...,Women,Tops & Blouses,Blouse
3,3,leather hors statu,1,None,3.583519,1,new tag leather hors retail stand foot high so...,Home,Home Décor,Home Décor Accents
4,4,24k gold plate rose,1,None,3.806662,0,complet certif authent,Women,Jewelry,Necklaces


In [0]:
all_df.general_category = all_df.general_category.astype('category')
all_df.sub_cat_1 = all_df.sub_cat_1.astype('category')
all_df.sub_cat_2 = all_df.sub_cat_2.astype('category')
all_df.brand_name = all_df.brand_name.astype('category')
all_df.item_condition_id = all_df.item_condition_id.astype('category')
all_df.shipping = all_df.shipping.astype('category')

In [0]:
from __future__ import print_function, division, absolute_import
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import column_or_1d
#from sklearn.preprocessing.label import _check_numpy_unicode_bug
import numpy as np
import pandas as pd
#from skutil.base import BaseSkutil
#from skutil.utils import validate_is_pd

__all__ = [
    'SafeLabelEncoder',
    'OneHotCategoricalEncoder'
]


def _get_unseen():
    """Basically just a static method
    instead of a class attribute to avoid
    someone accidentally changing it."""
    return 99999


class SafeLabelEncoder(LabelEncoder):
    """An extension of LabelEncoder that will
    not throw an exception for unseen data, but will
    instead return a default value of 99999

    Attributes
    ----------

    classes_ : the classes that are encoded
    """

    def transform(self, y):
        """Perform encoding if already fit.

        Parameters
        ----------

        y : array_like, shape=(n_samples,)
            The array to encode

        Returns
        -------

        e : array_like, shape=(n_samples,)
            The encoded array
        """
        check_is_fitted(self, 'classes_')
        y = column_or_1d(y, warn=True)

        classes = np.unique(y)
        #_check_numpy_unicode_bug(classes)

        # Check not too many:
        unseen = _get_unseen()
        if len(classes) >= unseen:
            raise ValueError('Too many factor levels in feature. Max is %i' % unseen)

        e = np.array([
                         np.searchsorted(self.classes_, x) if x in self.classes_ else unseen
                         for x in y
                         ])

        return e

## LightGBM + CatBoost

In [0]:
x_train, x_test, y_train, y_test = train_test_split(all_df.drop(columns=['train_id','price'], axis=1), \
                                                                     all_df['price'],\
                                                                    test_size = 0.2)

In [0]:
name_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features = 2 ** 17)
x_train_name_tfidf = name_tfidf_vectorizer.fit_transform(x_train.name)
x_test_name_tfidf = name_tfidf_vectorizer.transform(x_test.name)
name_standerdizer = StandardScaler(with_mean=False)
x_train_name_tfidf = name_standerdizer.fit_transform(x_train_name_tfidf)
x_test_name_tfidf = name_standerdizer.transform(x_test_name_tfidf)
pickle.dump(x_train_name_tfidf, open('/content/gdrive/My Drive/MerCari/x_train_name_tfidf.p','wb'))
pickle.dump(x_test_name_tfidf, open('/content/gdrive/My Drive/MerCari/x_test_name_tfidf.p','wb'))

desc_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features = 2 * (2 ** 17))
x_train_desc_tfidf = desc_tfidf_vectorizer.fit_transform(x_train.item_description)
x_test_desc_tfidf = desc_tfidf_vectorizer.transform(x_test.item_description)
desc_standerdizer = StandardScaler(with_mean=False)
x_train_desc_tfidf = desc_standerdizer.fit_transform(x_train_desc_tfidf)
x_test_desc_tfidf = desc_standerdizer.transform(x_test_desc_tfidf)

pickle.dump(x_train_desc_tfidf, open('/content/gdrive/My Drive/MerCari/x_train_desc_tfidf.p','wb'))
pickle.dump(x_test_desc_tfidf, open('/content/gdrive/My Drive/MerCari/x_test_desc_tfidf.p','wb'))

In [0]:
# load target data
y_train = pickle.load(open('/content/gdrive/My Drive/MerCari/y_train.p', 'rb'))
y_test = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test.p', 'rb'))

In [0]:
x_train_name_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_name_tfidf.p','rb'))
x_test_name_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_name_tfidf.p','rb'))
x_train_desc_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_desc_tfidf.p','rb'))
x_test_desc_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_desc_tfidf.p','rb'))

In [0]:
x_train_text_features = hstack([x_train_name_tfidf, x_train_desc_tfidf]).tocsr()
x_test_text_features = hstack([x_test_name_tfidf, x_test_desc_tfidf]).tocsr()

In [0]:
lgb_train = lgb.Dataset(x_train_text_features, y_train)
lgb_test = lgb.Dataset(x_test_text_features, y_test, reference=lgb_train)

In [0]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 100,
    'learning_rate': 0.7,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 3,
    'verbose': 0
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=150,
                valid_sets=lgb_test,
                early_stopping_rounds=10
                )
y_test_text_features_lgbm = gbm.predict(x_test_text_features, num_iteration=gbm.best_iteration)

print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

[1]	valid_0's rmse: 0.671099
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's rmse: 0.639879
[3]	valid_0's rmse: 0.619844
[4]	valid_0's rmse: 0.606384
[5]	valid_0's rmse: 0.598436
[6]	valid_0's rmse: 0.591338
[7]	valid_0's rmse: 0.585234
[8]	valid_0's rmse: 0.580418
[9]	valid_0's rmse: 0.576076
[10]	valid_0's rmse: 0.572438
[11]	valid_0's rmse: 0.569129
[12]	valid_0's rmse: 0.566324
[13]	valid_0's rmse: 0.563706
[14]	valid_0's rmse: 0.561106
[15]	valid_0's rmse: 0.558954
[16]	valid_0's rmse: 0.55704
[17]	valid_0's rmse: 0.55523
[18]	valid_0's rmse: 0.553666
[19]	valid_0's rmse: 0.552305
[20]	valid_0's rmse: 0.551061
[21]	valid_0's rmse: 0.549721
[22]	valid_0's rmse: 0.548096
[23]	valid_0's rmse: 0.547075
[24]	valid_0's rmse: 0.545961
[25]	valid_0's rmse: 0.544778
[26]	valid_0's rmse: 0.543771
[27]	valid_0's rmse: 0.542821
[28]	valid_0's rmse: 0.54189
[29]	valid_0's rmse: 0.540964
[30]	valid_0's rmse: 0.539985
[31]	valid_0's rmse: 0.53921
[32]	valid_0's rmse: 

In [0]:
pickle.dump(y_train_text_features_lgbm, open('/content/gdrive/My Drive/MerCari/y_train_text_features_lgbm.p','wb'))
pickle.dump(y_test_text_features_lgbm, open('/content/gdrive/My Drive/MerCari/y_test_text_features_lgbm.p','wb'))

In [0]:
y_train_text_features_lgbm.shape

(1184672,)

In [0]:
x_train_cat = pd.DataFrame()
x_train_cat['lgbm_text'] = y_train_text_features_lgbm

x_test_cat = pd.DataFrame()
x_test_cat['lgbm_text'] = y_test_text_features_lgbm
x_train_cat['item_condition_id'] = x_train.item_condition_id.values
x_train_cat['shipping'] = x_train.shipping.values
x_train_cat['brand_name'] = x_train.brand_name.values
x_train_cat['gen_cat'] = x_train.general_category.values
x_train_cat['sub_cat_1'] = x_train.sub_cat_1.values
x_train_cat['sub_cat_2'] = x_train.sub_cat_2.values

x_test_cat['item_condition_id'] = x_test.item_condition_id.values
x_test_cat['shipping'] = x_test.shipping.values
x_test_cat['brand_name'] = x_test.brand_name.values
x_test_cat['gen_cat'] = x_test.general_category.values
x_test_cat['sub_cat_1'] = x_test.sub_cat_1.values
x_test_cat['sub_cat_2'] = x_test.sub_cat_2.values

sub_cat_2_le = SafeLabelEncoder()
x_train_cat['sub_cat_2'] = sub_cat_2_le.fit_transform(x_train_cat.sub_cat_2.values)
x_test_cat['sub_cat_2'] = sub_cat_2_le.transform(x_test_cat.sub_cat_2.values)

sub_cat_1_le = SafeLabelEncoder()
x_train_cat['sub_cat_1'] = sub_cat_1_le.fit_transform(x_train_cat.sub_cat_1.values)
x_test_cat['sub_cat_1'] = sub_cat_1_le.transform(x_test_cat.sub_cat_1.values)

gen_cat_le = SafeLabelEncoder()
x_train_cat['gen_cat'] = gen_cat_le.fit_transform(x_train_cat.gen_cat.values)
x_test_cat['gen_cat'] = gen_cat_le.transform(x_test_cat.gen_cat.values)

shipping_le = SafeLabelEncoder()
x_train_cat['shipping'] = shipping_le.fit_transform(x_train_cat.shipping.values)
x_test_cat['shipping'] = shipping_le.transform(x_test_cat.shipping.values)

item_condition_id_le = SafeLabelEncoder()
x_train_cat['item_condition_id'] = item_condition_id_le.fit_transform(x_train_cat.item_condition_id.values)
x_test_cat['item_condition_id'] = item_condition_id_le.transform(x_test_cat.item_condition_id.values)

brand_name_le = SafeLabelEncoder()
x_train_cat['brand_name'] = brand_name_le.fit_transform(x_train_cat.brand_name.values)
x_test_cat['brand_name'] = brand_name_le.transform(x_test_cat.brand_name.values)

In [0]:
cat_params = {
    'loss_function' : 'RMSE',
    'eval_metric' : 'RMSE',
    'n_estimators' : 1000,
    'learning_rate' : 0.3,
    'bootstrap_type' : 'Bernoulli',
    'use_best_model' : True,
    'depth' : 10,
    'logging_level' : 'Verbose',
    'metric_period' : 3,
}

model=CatBoostRegressor(**cat_params)
model.fit(x_train_cat.values, y_train, cat_features=[1,2,3,4,5,6], eval_set=(x_test_cat.values , y_test), plot=True)
y_pred_cat_boost = model.predict(x_test_cat.values)

print('The rmse of prediction is:', mean_squared_error(y_test, y_pred_cat_boost) ** 0.5)

*The RMSE of prediction is 0.4709858*

In [0]:
y_train_cat_boost = model.predict(x_train_cat.values)
pickle.dump(y_train_cat_boost, open('/content/gdrive/My Drive/MerCari/y_train_lgbm_cat_boost.p','wb'))
pickle.dump(y_pred_cat_boost, open('/content/gdrive/My Drive/MerCari/y_test_lgbm_cat_boost.p','wb'))

In [0]:
brand_name_label_binarizer = LabelBinarizer(sparse_output=True)
x_train_brand_ohe = brand_name_label_binarizer.fit_transform(x_train.brand_name)
x_test_brand_ohe = brand_name_label_binarizer.transform(x_test.brand_name)
pickle.dump(x_train_brand_ohe, open('/content/gdrive/My Drive/MerCari/x_train_brand_ohe.p','wb'))
pickle.dump(x_test_brand_ohe, open('/content/gdrive/My Drive/MerCari/x_test_brand_ohe.p','wb'))

shipping_label_binarizer = LabelBinarizer(sparse_output=True)
x_train_shipping_ohe = shipping_label_binarizer.fit_transform(x_train.shipping)
x_test_shipping_ohe = shipping_label_binarizer.transform(x_test.shipping)
pickle.dump(x_train_shipping_ohe, open('/content/gdrive/My Drive/MerCari/x_train_shipping_ohe.p','wb'))
pickle.dump(x_test_shipping_ohe, open('/content/gdrive/My Drive/MerCari/x_test_shipping_ohe.p','wb'))

item_condition_id_binarizer = LabelBinarizer(sparse_output=True)
x_train_item_condition_ohe = item_condition_id_binarizer.fit_transform(x_train.item_condition_id)
x_test_item_condition_ohe = item_condition_id_binarizer.transform(x_test.item_condition_id)
pickle.dump(x_train_item_condition_ohe, open('/content/gdrive/My Drive/MerCari/x_train_item_condition_ohe.p','wb'))
pickle.dump(x_test_item_condition_ohe, open('/content/gdrive/My Drive/MerCari/x_test_item_condition_ohe.p','wb'))

general_category_binarizer = LabelBinarizer(sparse_output=True)
x_train_general_category_ohe = general_category_binarizer.fit_transform(x_train.general_category)
x_test_general_category_ohe = general_category_binarizer.transform(x_test.general_category)
pickle.dump(x_train_general_category_ohe, open('/content/gdrive/My Drive/MerCari/x_train_general_category_ohe.p','wb'))
pickle.dump(x_test_general_category_ohe, open('/content/gdrive/My Drive/MerCari/x_test_general_category_ohe.p','wb'))

sub_cat_1_binarizer = LabelBinarizer(sparse_output=True)
x_train_sub_cat_1_ohe = sub_cat_1_binarizer.fit_transform(x_train.sub_cat_1)
x_test_sub_cat_1_ohe = sub_cat_1_binarizer.transform(x_test.sub_cat_1)
pickle.dump(x_train_sub_cat_1_ohe, open('/content/gdrive/My Drive/MerCari/x_train_sub_cat_1_ohe.p','wb'))
pickle.dump(x_test_sub_cat_1_ohe, open('/content/gdrive/My Drive/MerCari/x_test_sub_cat_1_ohe.p','wb'))

sub_cat_2_binarizer = LabelBinarizer(sparse_output=True)
x_train_sub_cat_2_ohe = sub_cat_2_binarizer.fit_transform(x_train.sub_cat_2)
x_test_sub_cat_2_ohe = sub_cat_2_binarizer.transform(x_test.sub_cat_2)
pickle.dump(x_train_sub_cat_1_ohe, open('/content/gdrive/My Drive/MerCari/x_train_sub_cat_2_ohe.p','wb'))
pickle.dump(x_test_sub_cat_1_ohe, open('/content/gdrive/My Drive/MerCari/x_test_sub_cat_2_ohe.p','wb'))

In [0]:
x_train_brand_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_brand_ohe.p','rb'))
x_test_brand_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_brand_ohe.p','rb'))

x_train_shipping_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_shipping_ohe.p','rb'))
x_test_shipping_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_shipping_ohe.p','rb'))

x_train_item_condition_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_item_condition_ohe.p','rb'))
x_test_item_condition_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_item_condition_ohe.p','rb'))

x_train_general_category_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_general_category_ohe.p','rb'))
x_test_general_category_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_general_category_ohe.p','rb'))

x_train_sub_cat_1_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_sub_cat_1_ohe.p','rb'))
x_test_sub_cat_1_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_sub_cat_1_ohe.p','rb'))

x_train_sub_cat_2_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_sub_cat_2_ohe.p','rb'))
x_test_sub_cat_2_ohe = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_sub_cat_2_ohe.p','rb'))

## Count Vectorizer + TfidfVectorizer

In [0]:
name_bow_vectorizer = CountVectorizer(ngram_range=(1,2), max_features=2 ** 17)
x_train_name_bow = name_bow_vectorizer.fit_transform(list(x_train.name.values))
x_test_name_bow = name_bow_vectorizer.transform(list(x_test.name.values))
name_standerdizer = StandardScaler(with_mean=False)
x_train_name_bow = name_standerdizer.fit_transform(x_train_name_bow)
x_test_name_bow = name_standerdizer.transform(x_test_name_bow)
pickle.dump(x_train_name_bow, open('/content/gdrive/My Drive/MerCari/x_train_name_bow.p','wb'))
pickle.dump(x_test_name_bow, open('/content/gdrive/My Drive/MerCari/x_test_name_bow.p','wb'))

description_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3), max_features=2* (2 ** 17))
x_train_desc_tfidf = description_tfidf_vectorizer.fit_transform(list(x_train.item_description.values))
x_test_desc_tfidf = description_tfidf_vectorizer.transform(list(x_test.item_description.values))
description_standerdizer = StandardScaler(with_mean=False)
x_train_desc_tfidf = description_standerdizer.fit_transform(x_train_desc_tfidf)
x_test_desc_tfidf = description_standerdizer.transform(x_test_desc_tfidf)
pickle.dump(x_train_desc_tfidf, open('/content/gdrive/My Drive/MerCari/x_train_desc_tfidf.p','wb'))
pickle.dump(x_test_desc_tfidf, open('/content/gdrive/My Drive/MerCari/x_test_desc_tfidf.p','wb'))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [0]:

x_train_name_bow = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_name_bow.p','rb'))
x_test_name_bow = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_name_bow.p','rb'))
x_train_desc_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_desc_tfidf.p','rb'))
x_test_desc_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_desc_tfidf.p','rb'))

In [0]:
brand_name_bow_vectorizer = CountVectorizer()
x_train_brand_name_bow = brand_name_bow_vectorizer.fit_transform(list(x_train.brand_name.values))
x_test_brand_name_bow = brand_name_bow_vectorizer.transform(list(x_test.brand_name.values))
brand_name_standerdizer = StandardScaler(with_mean=False)
x_train_brand_name_bow = brand_name_standerdizer.fit_transform(x_train_brand_name_bow)
x_test_brand_name_bow = brand_name_standerdizer.transform(x_test_brand_name_bow)
pickle.dump(x_train_brand_name_bow, open('/content/gdrive/My Drive/MerCari/x_train_brand_name_bow.p','wb'))
pickle.dump(x_test_brand_name_bow, open('/content/gdrive/My Drive/MerCari/x_test_brand_name_bow.p','wb'))

general_category_bow_vectorizer = CountVectorizer()
x_train_general_category_bow = general_category_bow_vectorizer.fit_transform(list(x_train.general_category.values))
x_test_general_category_bow = general_category_bow_vectorizer.transform(list(x_test.general_category.values))
general_category_standerdizer = StandardScaler(with_mean=False)
x_train_general_category_bow = general_category_standerdizer.fit_transform(x_train_general_category_bow)
x_test_general_category_bow = general_category_standerdizer.transform(x_test_general_category_bow)
pickle.dump(x_train_general_category_bow, open('/content/gdrive/My Drive/MerCari/x_train_general_category_bow.p','wb'))
pickle.dump(x_test_general_category_bow, open('/content/gdrive/My Drive/MerCari/x_test_general_category_bow.p','wb'))

sub_cat_1_bow_vectorizer = CountVectorizer()
x_train_sub_cat_1_bow = sub_cat_1_bow_vectorizer.fit_transform(list(x_train.sub_cat_1.values))
x_test_sub_cat_1_bow = sub_cat_1_bow_vectorizer.transform(list(x_test.sub_cat_1.values))
sub_cat_1_standerdizer = StandardScaler(with_mean=False)
x_train_sub_cat_1_bow = sub_cat_1_standerdizer.fit_transform(x_train_sub_cat_1_bow)
x_test_sub_cat_1_bow = sub_cat_1_standerdizer.transform(x_test_sub_cat_1_bow)
pickle.dump(x_train_sub_cat_1_bow, open('/content/gdrive/My Drive/MerCari/x_train_sub_cat_1_bow.p','wb'))
pickle.dump(x_test_sub_cat_1_bow, open('/content/gdrive/My Drive/MerCari/x_test_sub_cat_1_bow.p','wb'))

sub_cat_2_bow_vectorizer = CountVectorizer()
x_train_sub_cat_2_bow = sub_cat_2_bow_vectorizer.fit_transform(list(x_train.sub_cat_2.values))
x_test_sub_cat_2_bow = sub_cat_2_bow_vectorizer.transform(list(x_test.sub_cat_2.values))
sub_cat_2_standerdizer = StandardScaler(with_mean=False)
x_train_sub_cat_2_bow = sub_cat_2_standerdizer.fit_transform(x_train_sub_cat_2_bow)
x_test_sub_cat_2_bow = sub_cat_2_standerdizer.transform(x_test_sub_cat_2_bow)
pickle.dump(x_train_sub_cat_2_bow, open('/content/gdrive/My Drive/MerCari/x_train_sub_cat_2_bow.p','wb'))
pickle.dump(x_test_sub_cat_2_bow, open('/content/gdrive/My Drive/MerCari/x_test_sub_cat_2_bow.p','wb'))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [0]:
x_train_bow_tfidf_ohe = hstack([x_train_name_bow,
                        x_train_desc_tfidf,
                        x_train_brand_name_bow,
                        x_train_shipping_ohe,
                        x_train_item_condition_ohe,
                        x_train_general_category_bow,
                        x_train_sub_cat_1_bow,
                        x_train_sub_cat_2_bow]).tocsr()

x_test_bow_tfidf_ohe = hstack([x_test_name_bow,
                        x_test_desc_tfidf,
                        x_test_brand_name_bow,
                        x_test_shipping_ohe,
                        x_test_item_condition_ohe,
                        x_test_general_category_bow,
                        x_test_sub_cat_1_bow,
                        x_test_sub_cat_2_bow]).tocsr()

In [0]:
model = Ridge(solver='auto',
        fit_intercept=True,
        alpha=0.1,
        max_iter=100,
        normalize=False,
        tol=0.05)
model.fit(x_train_bow_tfidf_ohe, y_train)
y_test_bow_tfidf_ohe_ridge = model.predict(x_test_bow_tfidf_ohe)
print('The rmse of prediction is:', mean_squared_error(y_test, y_test_bow_tfidf_ohe_ridge) ** 0.5)

The rmse of prediction is: 0.4819474774572451


In [0]:
y_test_lgbm_cat_boost = pickle.load(open('/content/gdrive/My Drive/y_test_lgbm_cat_boost.p','rb'))

In [0]:
pickle.dump(y_test_bow_tfidf_ohe_ridge, open('/content/gdrive/My Drive/MerCari/y_test_bow_tfidf_ohe_ridge.b','wb'))

In [0]:
y_test_bow_tfidf_ohe_ridge = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test_bow_tfidf_ohe_ridge.b','rb'))

In [0]:
y_pred = y_test_lgbm_cat_boost * 0.6 + y_test_bow_tfidf_ohe_ridge * 0.4
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 0.45093465480150213


In [0]:


sub_cat_2_le = SafeLabelEncoder()
x_train_sub_cat_2 = csr_matrix(sub_cat_2_le.fit_transform(x_train.sub_cat_2.values.tolist()).reshape(x_train.shape[0], -1))
x_test_sub_cat_2 =  csr_matrix(sub_cat_2_le.transform(x_test.sub_cat_2.values.tolist()).reshape(x_test.shape[0], -1))

sub_cat_1_le = SafeLabelEncoder()
x_train_sub_cat_1 =  csr_matrix(sub_cat_1_le.fit_transform(x_train.sub_cat_1.values.tolist()).reshape(x_train.shape[0], -1))
x_test_sub_cat_1 =  csr_matrix(sub_cat_1_le.transform(x_test.sub_cat_1.values.tolist()).reshape(x_test.shape[0], -1))

gen_cat_le = SafeLabelEncoder()
x_train_gen_cat =  csr_matrix(gen_cat_le.fit_transform(x_train.general_category.values.tolist()).reshape(x_train.shape[0], -1))
x_test_gen_cat =  csr_matrix(gen_cat_le.transform(x_test.general_category.values.tolist()).reshape(x_test.shape[0], -1))

brand_name_le = SafeLabelEncoder()
x_train_brand_name =  csr_matrix(brand_name_le.fit_transform(x_train.brand_name.values.tolist()).reshape(x_train.shape[0], -1))
x_test_brand_name =  csr_matrix(brand_name_le.transform(x_test.brand_name.values.tolist()).reshape(x_test.shape[0], -1))

In [0]:
x_train_lgbm_bow_tfidf_label_item_ohe = hstack([x_train_name_bow,
                                               x_train_desc_tfidf,
                                               x_train_item_condition_ohe,
                                               x_train_shipping_ohe,
                                               x_train_brand_name,
                                               x_train_gen_cat,
                                               x_train_sub_cat_1,
                                               x_train_sub_cat_2]).tocsr()

x_test_lgbm_bow_tfidf_label_item_ohe = hstack([x_test_name_bow,
                                               x_test_desc_tfidf,
                                               x_test_item_condition_ohe,
                                               x_test_shipping_ohe,
                                               x_test_brand_name,
                                               x_test_gen_cat,
                                               x_test_sub_cat_1,
                                               x_test_sub_cat_2]).tocsr()

In [0]:
lgb_train = lgb.Dataset(x_train_lgbm_bow_tfidf_label_item_ohe, y_train)
lgb_test = lgb.Dataset(x_test_lgbm_bow_tfidf_label_item_ohe, y_test, reference=lgb_train)

In [0]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 100,
    'learning_rate': 0.6,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 3,
    'verbose': 0
}

last_column = x_train_lgbm_bow_tfidf_label_item_ohe.shape[1] - 1

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=150,
                valid_sets=lgb_test,
                early_stopping_rounds=10,
                categorical_feature= [last_column, last_column-1, last_column-2,  last_column-3]
                )
y_test_lgbm_bow_tfidf_label_item_ohe = gbm.predict(x_test_lgbm_bow_tfidf_label_item_ohe, num_iteration=gbm.best_iteration)

print('The rmse of prediction is:', mean_squared_error(y_test, y_test_lgbm_bow_tfidf_label_item_ohe) ** 0.5)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [393222, 393223, 393224, 393225]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's rmse: 0.603239
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's rmse: 0.565394
[3]	valid_0's rmse: 0.547267
[4]	valid_0's rmse: 0.53699
[5]	valid_0's rmse: 0.529435
[6]	valid_0's rmse: 0.523389
[7]	valid_0's rmse: 0.518986
[8]	valid_0's rmse: 0.515266
[9]	valid_0's rmse: 0.511929
[10]	valid_0's rmse: 0.50876
[11]	valid_0's rmse: 0.506288
[12]	valid_0's rmse: 0.504072
[13]	valid_0's rmse: 0.501998
[14]	valid_0's rmse: 0.500324
[15]	valid_0's rmse: 0.498856
[16]	valid_0's rmse: 0.497459
[17]	valid_0's rmse: 0.496094
[18]	valid_0's rmse: 0.494845
[19]	valid_0's rmse: 0.49318
[20]	valid_0's rmse: 0.49187
[21]	valid_0's rmse: 0.491013
[22]	valid_0's rmse: 0.490113
[23]	valid_0's rmse: 0.489123
[24]	valid_0's rmse: 0.488307
[25]	valid_0's rmse: 0.487455
[26]	valid_0's rmse: 0.486728
[27]	valid_0's rmse: 0.486199
[28]	valid_0's rmse: 0.485518
[29]	valid_0's rmse: 0.484984
[30]	valid_0's rmse: 0.484412
[31]	valid_0's rmse: 0.48367
[32]	valid_0's rmse: 0

In [0]:
y_pred = (y_test_lgbm_cat_boost + y_test_bow_tfidf_ohe_ridge + y_test_lgbm_bow_tfidf_label_item_ohe)/3
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 0.44113609191779046


## FTRL + Word bag

In [0]:
wb = wordbatch.WordBatch(extractor=(WordBag, {"hash_ngrams": 2,
                                                              "hash_ngrams_weights": [1.5, 1.0],
                                                              "norm": None,
                                                              "tf": 'binary',
                                                              "idf": None,
                                                              }), procs=8)
x_train_name_wb = wb.fit_transform(x_train.name)
x_test_name_wb = wb.transform(x_test.name)

Normalize text
Normalize text


In [0]:
wb = wordbatch.WordBatch(extractor=(WordBag, {"hash_ngrams": 2,
                                                              "hash_ngrams_weights": [1.0, 1.0],
                                                              "norm": "l2",
                                                              "tf": 1.0,
                                                              "idf": None}), procs=8)
x_train_desc_wb = wb.fit_transform(x_train.item_description)
x_test_desc_wb = wb.transform(x_test.item_description)

Normalize text
Normalize text


In [0]:
pickle.dump(x_train_name_wb, open('/content/gdrive/My Drive/MerCari/x_train_name_wb.p','wb'))
pickle.dump(x_test_name_wb, open('/content/gdrive/My Drive/MerCari/x_test_name_wb.p','wb'))

pickle.dump(x_train_desc_wb, open('/content/gdrive/My Drive/MerCari/x_train_desc_wb.p','wb'))
pickle.dump(x_test_desc_wb, open('/content/gdrive/My Drive/MerCari/x_test_desc_wb.p','wb'))

In [0]:
x_train_name_wb.shape

(1184672, 10000000)

In [0]:
x_train_sub_cat_1_bow  =pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_sub_cat_1_bow.p','rb'))
x_test_sub_cat_1_bow = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_sub_cat_1_bow.p','rb'))

x_train_sub_cat_2_bow  =pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_sub_cat_2_bow.p','rb'))
x_test_sub_cat_2_bow = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_sub_cat_2_bow.p','rb'))

x_train_general_category_bow  =pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_general_category_bow.p','rb'))
x_test_general_category_bow = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_general_category_bow.p','rb'))

In [0]:
x_train = hstack([x_train_name_wb, 
                  x_train_desc_wb,
                  x_train_brand_ohe,
                  x_train_item_condition_ohe,
                  x_train_shipping_ohe,
                  x_train_general_category_bow,
                  x_train_sub_cat_1_bow,
                  x_train_sub_cat_2_bow
                 ]).tocsr()

x_test = hstack([x_test_name_wb, 
                  x_test_desc_wb,
                  x_test_brand_ohe,
                  x_test_item_condition_ohe,
                  x_test_shipping_ohe,
                  x_test_general_category_bow,
                  x_test_sub_cat_1_bow,
                  x_test_sub_cat_2_bow
                 ]).tocsr()

In [0]:
x_train_wb = hstack([x_train_name_wb, x_train_desc_wb]).tocsr()
x_test_wb = hstack([x_test_name_wb, x_test_desc_wb]).tocsr()

In [0]:
model_ridge = Ridge(alpha=0.4, tol=0.01, max_iter=300)

y_ridge = model_ridge.fit(x_train_wb, y_train).predict(x_test_wb)
print('The rmse of prediction is:', mean_squared_error(y_test, y_ridge) ** 0.5)

The rmse of prediction is: 0.49278020080967255


In [0]:
model_ftrl = FTRL(alpha=0.01, beta=0.1, L1=0.00001, L2=1.0, D=x_test.shape[1], \
                  iters=300, inv_link="identity", threads=1)
y_ftrl_pred = model_ftrl.fit(x_train, y_train).predict(x_test)
pickle.dump(y_ftrl_pred, open('/content/gdrive/My Drive/MerCari/y_ftrl_pred.p','wb'))
print('The rmse of prediction is:', mean_squared_error(y_test, y_ftrl_pred) ** 0.5)

Total e: 458204.68059433124
Total e: 414845.9094758803
Total e: 397939.4154792284
Total e: 386657.26692061493
Total e: 377640.62445034296
Total e: 370330.7357211696
Total e: 363966.50160679617
Total e: 358466.407986406
Total e: 353467.044886979
Total e: 349001.5900117263
Total e: 344875.32845564
Total e: 341106.82591397915
Total e: 337568.64444464596
Total e: 334298.7032346241
Total e: 331193.64892632706
Total e: 328294.9617081796
Total e: 325529.89978537674
Total e: 322926.2680255637
Total e: 320427.11173325905
Total e: 318065.9255445491
Total e: 315783.6085346258
Total e: 313624.16873555735
Total e: 311523.1389697432
Total e: 309522.76817173697
Total e: 307574.0130754293
Total e: 305718.52971685905
Total e: 303906.3039629107
Total e: 302175.4278216974
Total e: 300479.5678070104
Total e: 298852.8210234505
Total e: 297257.35050631204
Total e: 295724.9901364896
Total e: 294219.23624200036
Total e: 292770.4606125175
Total e: 291345.14474793617
Total e: 289971.4137417834
Total e: 288618.2

In [0]:
y_pred = y_test_lgbm_cat_boost * 0.5 + y_ftrl_pred * 0.5
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 0.4408236787764176


## Conclusion / Summary of the Approach


### LightGBM+CatBoost
1. We are going to get two pedictions from two different models
and take the average of them
2. Tfidf vectorize(1,2) name and item description -> Run a lightGBM regression on these vectorize and get the predictions of train and test data.
3. Label Encode BrandName, GeneralCategory, SubCat1, SubCat2, ItemCondition and Shipping.
4. Pass the predictions from LightGBM along with the label encoded features to CatBoost. Get the predictions for train and test data. Lets call this ** prediction number 1**. This is the one of the predictions we are going to average from.
---------------------------------------------------------------
### TFIDF+CountVectorizer, Ridge and LightGBM
1. Name - Count vectorize(1,2)
Description - TFIDF vectorize(1,3)
Brand, general category, sub category 1, sub category 2 - BOW
Shipping, item condition - Binarize
2. Pass these features to RidgeRegression models. Lets call this **prediction number 2**.
3. Average the LightGBM+CatBoost predictions(**1**) and Ridge Predictions(**2**) -> **0.4509** 
4. Name - Count vectorize(1,2)
Description - TFIDF vectorize(1,3)
Brand, generalCategory, subCategory 1, subCategory 2 - LabelEncode
Shipping, item condition - Binarize
5. Pass these features to LightGBM and get the predictions. **Prediction number 3**
6. Average the predictions of LightGBM+CatBoost(**1**), Ridge(**2**), LightGBM(**3**) -> **0.441136**
---------------------------------------------------------------
### WordBacth+FTRL
1. Name - WordBatch using WordBag(1,2)
Desc - WordBatch using WordBag(1,2)
Brand, ItemCondition and Shipping - Binarize
GeneralCategory, SubCat1, SubCat2 - LabelEncode
2. Apply FTRL on these features and get predictions.
Prediction number 4
3. Average LightGBM+CatBoost(**1**) and FTRL(**4**) -> **0.44082**

# Custom Features + Things I tried but did not work out

In [0]:
# https://pbpython.com/pandas_transform.html -> Good explaination about the transform function
all_df['name_count'] = all_df.groupby('name')['name'].transform('count')
all_df['gen_cat_count'] = all_df.groupby('general_category')['general_category'].transform('count')
all_df['sub_cat_1_count'] = all_df.groupby('sub_cat_1')['sub_cat_1'].transform('count')
all_df['sub_cat_2_count'] = all_df.groupby('sub_cat_2')['sub_cat_2'].transform('count')
all_df['brand_name_count'] = all_df.groupby('brand_name')['brand_name'].transform('count')

all_df['name_len'] = all_df['name'].apply(lambda x: len(x))
all_df['des_len'] = all_df['item_description'].apply(lambda x: len(x))
all_df['name_desc_len_ratio'] = all_df['name_len']/all_df['des_len']
all_df['desc_word_count'] = all_df['item_description'].apply(lambda x: len(x.split()))
all_df['mean_des'] = all_df['item_description'].apply(lambda x: 0 if len(x) == 0 else float(len(x.split())) / len(x)) * 10
all_df['name_word_count'] = all_df['name'].apply(lambda x: len(x.split()))
all_df['mean_name'] = all_df['name'].apply(lambda x: 0 if len(x) == 0 else float(len(x.split())) / len(x)) * 10
all_df['desc_letters_per_word'] = all_df['des_len'] / all_df['desc_word_count']
all_df['name_letters_per_word'] = all_df['name_len'] / all_df['name_word_count']
all_df['NameDigitCount'] = all_df.name.str.count('[0-9]')
all_df['DescriptionDigitCount'] = all_df.item_description.str.count('[0-9]')
all_df['NameDigitCountRatio'] = all_df['NameDigitCount'] / all_df['name_word_count']
all_df['DescriptionDigitCountRatio'] = all_df['DescriptionDigitCount']/all_df['desc_word_count']

In [0]:
pickle.dump(all_df, open('/content/gdrive/My Drive/MerCari/all_df_with_custom_featuresw.p','wb'))

In [0]:
x_train, x_test, y_train, y_test = train_test_split(all_df.drop(columns=['train_id','price'], axis=1), \
                                                                     all_df['price'],\
                                                                    test_size = 0.2)

In [0]:
pickle.dump(x_train, open('/content/gdrive/My Drive/MerCari/x_train_raw.p','wb'))
pickle.dump(x_test, open('/content/gdrive/My Drive/MerCari/x_test_raw.p','wb'))
pickle.dump(y_train, open('/content/gdrive/My Drive/MerCari/y_train.p','wb'))
pickle.dump(y_test, open('/content/gdrive/My Drive/MerCari/y_test.p','wb'))

In [0]:
x_train = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_raw.p','rb'))
x_test = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_raw.p','rb'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 'pandas.core' is private. Use 'pandas.Categorical'
  """Entry point for launching an IPython kernel.


In [0]:
x_train.columns

Index(['name', 'item_condition_id', 'brand_name', 'shipping',
       'item_description', 'general_category', 'sub_cat_1', 'sub_cat_2'],
      dtype='object')

In [0]:
class TargetEncoder:
    # Adapted from https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features
    def __repr__(self):
        return 'TargetEncoder'

    def __init__(self, cols, smoothing=1, min_samples_leaf=1, noise_level=0, keep_original=False):
        self.cols = cols
        self.smoothing = smoothing
        self.min_samples_leaf = min_samples_leaf
        self.noise_level = noise_level
        self.keep_original = keep_original

    @staticmethod
    def add_noise(series, noise_level):
        return series * (1 + noise_level * np.random.randn(len(series)))

    def encode(self, train, test, target):
        for col in self.cols:
            if self.keep_original:
                train[col + '_te'], test[col + '_te'] = self.encode_column(train[col], test[col], target)
            else:
                train[col], test[col] = self.encode_column(train[col], test[col], target)
        return train, test

    def encode_column(self, trn_series, tst_series, target):
        temp = pd.concat([trn_series, target], axis=1)
        # Compute target mean
        averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
        # Compute smoothing
        smoothing = 1 / (1 + np.exp(-(averages["count"] - self.min_samples_leaf) / self.smoothing))
        # Apply average function to all target data
        prior = target.mean()
        # The bigger the count the less full_avg is taken into account
        averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
        averages.drop(['mean', 'count'], axis=1, inplace=True)
        # Apply averages to trn and tst series
        ft_trn_series = pd.merge(
            trn_series.to_frame(trn_series.name),
            averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
            on=trn_series.name,
            how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
        # pd.merge does not keep the index so restore it
        ft_trn_series.index = trn_series.index
        ft_tst_series = pd.merge(
            tst_series.to_frame(tst_series.name),
            averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
            on=tst_series.name,
            how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
        # pd.merge does not keep the index so restore it
        ft_tst_series.index = tst_series.index
        return self.add_noise(ft_trn_series, self.noise_level), self.add_noise(ft_tst_series, self.noise_level)   


In [0]:
# load target data
y_train = pickle.load(open('/content/gdrive/My Drive/MerCari/y_train.p', 'rb'))
y_test = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test.p', 'rb'))

In [0]:
name_wb_vectorizer = wordbatch.WordBatch(extractor=( 
    WordBag, {"hash_ngrams" : 2, 
              "hash_ngrams_weights" : [1.5, 1.0],
              "hash_size" : 2 ** 17, 
              "norm" : None,
              "tf" : "binary",
              "idf" : None}
), procs=8)
x_train_wb_name = name_wb_vectorizer.fit_transform(x_train.name)
x_test_wb_name = name_wb_vectorizer.transform(x_test.name)

pickle.dump(x_train_wb_name, open('/content/gdrive/My Drive/MerCari/x_train_wb_name.p','wb'))
pickle.dump(x_test_wb_name, open('/content/gdrive/My Drive/MerCari/x_test_wb_name.p','wb'))

Normalize text
Normalize text


In [0]:
# load wordbatch data
x_train_wb_name = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_wb_name.p', 'rb')).tocsr()
x_test_wb_name = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_wb_name.p', 'rb')).tocsr()

In [0]:
x_train_wb_name = normalize(x_train_wb_name, axis=0)
x_test_wb_name = normalize(x_test_wb_name, axis=0)

In [0]:
model_ridge = Ridge(solver = 'sag', fit_intercept = True, random_state=205, alpha=2.8, max_iter=100)
model_ridge.fit(x_train_wb_name, y_train)
y_test_wb_name = model_ridge.predict(x_test_wb_name)

print('The rmse of prediction is:', mean_squared_error(y_test, y_test_wb_name) ** 0.5)

The rmse of prediction is: 0.5726734635285611


In [0]:
y_train_wb_name = model_ridge.predict(x_train_wb_name)

pickle.dump(y_train_wb_name, open('/content/gdrive/My Drive/MerCari/y_train_wb_name.p','wb'))
pickle.dump(y_test_wb_name, open('/content/gdrive/My Drive/MerCari/y_test_wb_name.p','wb'))

In [0]:
y_train_wb_name = pickle.load(open('/content/gdrive/My Drive/MerCari/y_train_wb_name.p', 'rb'))
y_test_wb_name = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test_wb_name.p', 'rb'))

In [0]:
desc_wb_vectorizer = wordbatch.WordBatch(extractor=( 
    WordBag, {"hash_ngrams" : 2, 
              "hash_ngrams_weights" : [1.0, 1.0],
              "hash_size" : 2 * 2 ** 17, 
              "norm" : "l2",
              "tf" : 1.0,
              "idf" : None}
), procs=8)
x_train_wb_desc = desc_wb_vectorizer.fit_transform(x_train.item_description)
x_test_wb_desc = desc_wb_vectorizer.transform(x_test.item_description)

pickle.dump(x_train_wb_desc, open('/content/gdrive/My Drive/MerCari/x_train_wb_desc.p','wb'))
pickle.dump(x_test_wb_desc, open('/content/gdrive/My Drive/MerCari/x_test_wb_desc.p','wb'))

Normalize text
Normalize text


In [0]:
x_train_wb_desc = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_wb_desc.p', 'rb')).tocsr()
x_test_wb_desc = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_wb_desc.p', 'rb')).tocsr()

In [0]:
x_train_wb_desc = normalize(x_train_wb_desc, axis=0)
x_test_wb_desc = normalize(x_test_wb_desc, axis=0)

In [0]:
model_ridge_desc = Ridge(random_state=205, alpha=5, max_iter=100)
model_ridge_desc.fit(x_train_wb_desc, y_train)
y_test_wb_desc = model_ridge_desc.predict(x_test_wb_desc)

print('The rmse of prediction is:', mean_squared_error(y_test, y_test_wb_desc) ** 0.5)

The rmse of prediction is: 0.610455683382548


In [0]:
y_train_wb_desc = model_ridge_desc.predict(x_train_wb_desc)

pickle.dump(y_train_wb_desc, open('/content/gdrive/My Drive/MerCari/y_train_wb_desc.p','wb'))
pickle.dump(y_test_wb_desc, open('/content/gdrive/My Drive/MerCari/y_test_wb_desc.p','wb'))

In [0]:
y_train_wb_desc = pickle.load(open('/content/gdrive/My Drive/MerCari/y_train_wb_desc.p', 'rb'))
y_test_wb_desc = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test_wb_desc.p', 'rb'))

In [0]:
x_train_wb_ohe = hstack([x_train_wb_name,
                        x_train_wb_desc,
                        x_train_brand_ohe,
                        x_train_shipping_ohe,
                        x_train_item_condition_ohe,
                        x_train_general_category_ohe,
                        x_train_sub_cat_1_ohe,
                        x_train_sub_cat_2_ohe])

x_test_wb_ohe = hstack([x_test_wb_name,
                        x_test_wb_desc,
                        x_test_brand_ohe,
                        x_test_shipping_ohe,
                        x_test_item_condition_ohe,
                        x_test_general_category_ohe,
                        x_test_sub_cat_1_ohe,
                        x_test_sub_cat_2_ohe])

In [0]:
pickle.dump(x_train_wb_ohe, open('/content/gdrive/My Drive/MerCari/x_train_wb_ohe.p','wb'))
pickle.dump(x_test_wb_ohe, open('/content/gdrive/My Drive/MerCari/x_test_wb_ohe.p','wb'))

In [0]:
model_fm_ftrl = FM_FTRL(alpha=0.035, beta=0.001, L1=0.00001, L2=0.15, D=x_train_wb_ohe.shape[1],
                alpha_fm=0.05, L2_fm=0.0, init_fm=0.01,
                D_fm=100, e_noise=0, iters=12, inv_link="identity", threads=4)

model_fm_ftrl.fit(x_train_wb_ohe, y_train)

Total e: 496661.0600160163
Total e: 445692.14027547557
Total e: 425456.88092341396
Total e: 411512.7075939182
Total e: 400621.8184924071
Total e: 391543.3385823677
Total e: 383678.50618526305
Total e: 376672.85169254575
Total e: 370302.7678164642
Total e: 364419.8454385314
Total e: 358917.59641346306
Total e: 353716.9786092178


In [0]:
y_test_wb_ohe_fm_ftrl = model_fm_ftrl.predict(x_test_wb_ohe)

print('The rmse of prediction is:', mean_squared_error(y_test, y_test_wb_ohe_fm_ftrl) ** 0.5)

The rmse of prediction is: 0.58172310737566


In [0]:
y_train_wb_ohe_fm_ftrl = model_fm_ftrl.predict(x_train_wb_ohe)

pickle.dump(y_train_wb_ohe_fm_ftrl, open('/content/gdrive/My Drive/MerCari/y_train_wb_ohe_fm_ftrl.p','wb'))
pickle.dump(y_test_wb_ohe_fm_ftrl, open('/content/gdrive/My Drive/MerCari/y_test_wb_ohe_fm_ftrl.p','wb'))

In [0]:
y_train_wb_ohe_fm_ftrl = pickle.load(open('/content/gdrive/My Drive/MerCari/y_train_wb_ohe_fm_ftrl.p','rb'))
y_test_wb_ohe_fm_ftrl = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test_wb_ohe_fm_ftrl.p','rb'))

In [0]:
model_ftrl = FTRL(alpha=0.01, beta=0.1, L1=0.00001, L2=1.0, D=x_train_wb_ohe.shape[1], \
                  iters=100, inv_link="identity", threads=1)
model_ftrl.fit(x_train_wb_ohe.tocsr(), y_train)

Total e: 572035.8649561704
Total e: 536347.0288463333
Total e: 523846.7997082726
Total e: 515107.2599872324
Total e: 508239.7624315468
Total e: 502529.609763402
Total e: 497619.51946607284
Total e: 493301.4734013436
Total e: 489439.7917281098
Total e: 485941.01542402525
Total e: 482740.9010440825
Total e: 479791.7446561268
Total e: 477055.24476328236
Total e: 474502.22751452826
Total e: 472109.4243951017
Total e: 469856.66440833657
Total e: 467728.9527810174
Total e: 465712.9137041189
Total e: 463797.7162173173
Total e: 461974.0470159824
Total e: 460234.55097687675
Total e: 458571.696177722
Total e: 456979.46783204627
Total e: 455451.9098073967
Total e: 453984.2677255518
Total e: 452572.1682645232
Total e: 451211.9208350847
Total e: 449899.9810229217
Total e: 448633.27954885765
Total e: 447408.90593463636
Total e: 446224.54660245683
Total e: 445077.65906539094
Total e: 443965.5729616609
Total e: 442886.84076025686
Total e: 441839.64534386474
Total e: 440822.23092554125
Total e: 439833.

In [0]:
y_test_wb_ohe_ftrl = model_ftrl.predict(x_test_wb_ohe)
print('The rmse of prediction is:', mean_squared_error(y_test, y_test_wb_ohe_ftrl) ** 0.5)

The rmse of prediction is: 0.5452087778677622


In [0]:
y_train_wb_ohe_ftrl = model_ftrl.predict(x_train_wb_ohe)
pickle.dump(y_train_wb_ohe_ftrl, open('/content/gdrive/My Drive/MerCari/y_train_wb_ohe_ftrl.p','wb'))
pickle.dump(y_test_wb_ohe_ftrl, open('/content/gdrive/My Drive/MerCari/y_test_wb_ohe_ftrl.p','wb'))

In [0]:
y_train_wb_ohe_ftrl = pickle.load(open('/content/gdrive/My Drive/MerCari/y_train_wb_ohe_ftrl.p','rb'))
y_test_wb_ohe_ftrl = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test_wb_ohe_ftrl.p','rb'))

In [0]:
name_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features = 2 ** 17)
x_train_name_tfidf = name_tfidf_vectorizer.fit_transform(x_train.name)
x_test_name_tfidf = name_tfidf_vectorizer.transform(x_test.name)
name_standerdizer = StandardScaler(with_mean=False)
x_train_name_tfidf = name_standerdizer.fit_transform(x_train_name_tfidf)
x_test_name_tfidf = name_standerdizer.transform(x_test_name_tfidf)
pickle.dump(x_train_name_tfidf, open('/content/gdrive/My Drive/MerCari/x_train_name_tfidf.p','wb'))
pickle.dump(x_test_name_tfidf, open('/content/gdrive/My Drive/MerCari/x_test_name_tfidf.p','wb'))

desc_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features = 2 * (2 ** 17))
x_train_desc_tfidf = desc_tfidf_vectorizer.fit_transform(x_train.item_description)
x_test_desc_tfidf = desc_tfidf_vectorizer.transform(x_test.item_description)
desc_standerdizer = StandardScaler(with_mean=False)
x_train_desc_tfidf = desc_standerdizer.fit_transform(x_train_desc_tfidf)
x_test_desc_tfidf = desc_standerdizer.transform(x_test_desc_tfidf)

pickle.dump(x_train_desc_tfidf, open('/content/gdrive/My Drive/MerCari/x_train_desc_tfidf.p','wb'))
pickle.dump(x_test_desc_tfidf, open('/content/gdrive/My Drive/MerCari/x_test_desc_tfidf.p','wb'))

In [0]:
x_train_name_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_name_tfidf.p','rb'))
x_test_name_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_name_tfidf.p','rb'))
x_train_desc_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_desc_tfidf.p','rb'))
x_test_desc_tfidf = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_desc_tfidf.p','rb'))

In [0]:
x_train_tfidf_ohe = hstack([x_train_name_tfidf,
                        x_train_desc_tfidf,
                        x_train_brand_ohe,
                        x_train_shipping_ohe,
                        x_train_item_condition_ohe,
                        x_train_general_category_ohe,
                        x_train_sub_cat_1_ohe,
                        x_train_sub_cat_2_ohe]).tocsr()

x_test_tfidf_ohe = hstack([x_test_name_tfidf,
                        x_test_desc_tfidf,
                        x_test_brand_ohe,
                        x_test_shipping_ohe,
                        x_test_item_condition_ohe,
                        x_test_general_category_ohe,
                        x_test_sub_cat_1_ohe,
                        x_test_sub_cat_2_ohe]).tocsr()

In [0]:
model_tfidf_ohe = Ridge(random_state=205, alpha=4, max_iter=200)
model_tfidf_ohe.fit(x_train_tfidf_ohe, y_train)
y_test_tfidf_ohe_ridge = model_tfidf_ohe.predict(x_test_tfidf_ohe)
print('The rmse of prediction is:', mean_squared_error(y_test, y_test_tfidf_ohe_ridge) ** 0.5)

The rmse of prediction is: 0.5411923191381363


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [0]:
y_train_tfidf_ohe_ridge = model_tfidf_ohe.predict(x_train_tfidf_ohe)
pickle.dump(y_train_tfidf_ohe_ridge, open('/content/gdrive/My Drive/MerCari/y_train_tfidf_ohe_ridge.p','wb'))
pickle.dump(y_test_tfidf_ohe_ridge, open('/content/gdrive/My Drive/MerCari/y_test_tfidf_ohe_ridge.p','wb'))

In [0]:
y_train_tfidf_ohe_ridge = pickle.load(open('/content/gdrive/My Drive/MerCari/y_train_tfidf_ohe_ridge.p','rb'))
y_test_tfidf_ohe_ridge = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test_tfidf_ohe_ridge.p','rb'))

In [0]:
y_train.sample(5)

964037    3.713572
392971    3.044522
889260    2.890372
599506    3.178054
823825    3.295837
Name: price, dtype: float64

In [0]:
model_mnb = MultinomialNB(alpha=0.01)
model_mnb.fit(x_train_tfidf_ohe, y_train >= 4)
y_train_tfidf_ohe_mnb = model_mnb.predict_proba(x_train_tfidf_ohe)[:, 1]
y_test_tfidf_ohe_mnb = model_mnb.predict_proba(x_test_tfidf_ohe)[:, 1]
pickle.dump(y_train_tfidf_ohe_mnb, open('/content/gdrive/My Drive/MerCari/y_train_tfidf_ohe_mnb.p', 'wb'))
pickle.dump(y_test_tfidf_ohe_mnb, open('/content/gdrive/My Drive/MerCari/y_test_tfidf_ohe_mnb.p', 'wb'))

In [0]:
y_train_tfidf_ohe_mnb = pickle.load(open('/content/gdrive/My Drive/MerCari/y_train_tfidf_ohe_mnb.p','rb'))
y_test_tfidf_ohe_mnb = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test_tfidf_ohe_mnb.p','rb'))

In [0]:
fselect = SelectKBest(f_regression, k=50000)
x_train_wb_ohe_50k = fselect.fit_transform(x_train_wb_ohe, y_train)
x_test_wb_ohe_50k = fselect.transform(x_test_wb_ohe)

pickle.dump(x_train_wb_ohe_50k, open('/content/gdrive/My Drive/MerCari/x_train_wb_ohe_50k.p', 'wb'))
pickle.dump(x_test_wb_ohe_50k, open('/content/gdrive/My Drive/MerCari/x_test_wb_ohe_50k.p', 'wb'))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [0]:
x_train_wb_ohe_50k = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_wb_ohe_50k.p','rb'))
x_test_wb_ohe_50k = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_wb_ohe_50k.p','rb'))

In [0]:
categorical_features = ['brand_name', 'general_category', 'sub_cat_1', 'sub_cat_2']
target_encode = TargetEncoder(min_samples_leaf=100, smoothing=10, noise_level=0.01,
                              keep_original=True, cols=categorical_features)
x_train_target_encode, x_test_target_encode = target_encode.encode(x_train, x_test, y_train)

pickle.dump(x_train_target_encode, open('/content/gdrive/My Drive/MerCari/x_train_target_encode.p','wb'))
pickle.dump(x_test_target_encode, open('/content/gdrive/My Drive/MerCari/x_test_target_encode.p','wb'))

In [0]:
x_train_target_encode = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_target_encode.p','rb'))
x_test_target_encode = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_target_encode.p','rb'))

In [0]:
x_train_target_encode['name_ridge'] = y_train_wb_name
x_train_target_encode['desc_ridge'] = y_train_wb_desc
x_train_target_encode['ridge'] = y_train_tfidf_ohe_ridge
x_train_target_encode['mnb'] = y_train_tfidf_ohe_mnb

x_test_target_encode['name_ridge'] = y_test_wb_name
x_test_target_encode['desc_ridge'] = y_test_wb_desc
x_test_target_encode['ridge'] = y_test_tfidf_ohe_ridge
x_test_target_encode['mnb'] = y_test_tfidf_ohe_mnb

In [0]:
x_train_wb_ohe_50k.shape

(1184672, 50000)

In [0]:
x_train_target_encode.columns

Index(['name', 'item_condition_id', 'brand_name', 'shipping',
       'item_description', 'general_category', 'sub_cat_1', 'sub_cat_2',
       'name_count', 'gen_cat_count', 'sub_cat_1_count', 'sub_cat_2_count',
       'brand_name_count', 'name_len', 'des_len', 'name_desc_len_ratio',
       'desc_word_count', 'mean_des', 'name_word_count', 'mean_name',
       'desc_letters_per_word', 'name_letters_per_word', 'NameDigitCount',
       'DescriptionDigitCount', 'NameDigitCountRatio',
       'DescriptionDigitCountRatio', 'brand_name_te', 'general_category_te',
       'sub_cat_1_te', 'sub_cat_2_te', 'name_ridge', 'desc_ridge', 'ridge',
       'mnb'],
      dtype='object')

In [0]:
x_train_dummies = x_train_target_encode[['brand_name_te', 'general_category_te',
       'sub_cat_1_te', 'sub_cat_2_te', 'name_ridge', 'desc_ridge', 'ridge',
       'mnb']].astype(float).values

x_test_dummies = x_test_target_encode[['brand_name_te', 'general_category_te',
       'sub_cat_1_te', 'sub_cat_2_te', 'name_ridge', 'desc_ridge', 'ridge',
       'mnb']].astype(float).values

In [0]:
temp = [x_train_dummies, x_test_dummies]
del temp, x_train_dummies, x_test_dummies
import gc
gc.collect()

303

In [0]:
x_train_wb_ohe_50k_and_preds = hstack([x_train_wb_ohe_50k, csr_matrix(x_train_dummies)])
x_test_wb_ohe_50k_and_preds = hstack([x_test_wb_ohe_50k, csr_matrix(x_test_dummies)])

In [0]:
pickle.dump(x_train_wb_ohe_50k_and_preds, open('/content/gdrive/My Drive/MerCari/x_train_wb_ohe_50k_and_preds.p','wb'))
pickle.dump(x_test_wb_ohe_50k_and_preds, open('/content/gdrive/My Drive/MerCari/x_test_wb_ohe_50k_and_preds.p','wb'))

In [0]:
x_train_wb_ohe_50k_and_preds = pickle.load(open('/content/gdrive/My Drive/MerCari/x_train_wb_ohe_50k_and_preds.p','rb')).tocsr()
x_test_wb_ohe_50k_and_preds = pickle.load(open('/content/gdrive/My Drive/MerCari/x_test_wb_ohe_50k_and_preds.p','rb')).tocsr()

In [0]:
lgb_train = lgb.Dataset(x_train_wb_ohe_50k_and_preds, y_train)
lgb_eval = lgb.Dataset(x_test_wb_ohe_50k_and_preds, y_test, reference=lgb_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'RMSE',
    'max_depth' : 20,
    'num_leaves': 500,
    'learning_rate': 1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 3,
    'n_jobs' : -1,
    'verbose': 0,
    'lambda_l1': 10,
    'lambda_l2': 10
}
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=lgb_eval,
                early_stopping_rounds=20
                )

y_test_wb_ohe_50k_and_preds = gbm.predict(x_test_wb_ohe_50k_and_preds, num_iterations=gbm.best_iteration)

print('The rmse of prediction is:', mean_squared_error(y_test, y_test_wb_ohe_50k_and_preds) ** 0.5)

[1]	valid_0's rmse: 0.527515
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 0.521926
[3]	valid_0's rmse: 0.519757
[4]	valid_0's rmse: 0.520369
[5]	valid_0's rmse: 0.520016
[6]	valid_0's rmse: 0.519613
[7]	valid_0's rmse: 0.519285
[8]	valid_0's rmse: 0.519061
[9]	valid_0's rmse: 0.51874
[10]	valid_0's rmse: 0.518878
[11]	valid_0's rmse: 0.519868
[12]	valid_0's rmse: 0.519797
[13]	valid_0's rmse: 0.520057
[14]	valid_0's rmse: 0.520236
[15]	valid_0's rmse: 0.520014
[16]	valid_0's rmse: 0.520265
[17]	valid_0's rmse: 0.520059
[18]	valid_0's rmse: 0.520109
[19]	valid_0's rmse: 0.5203
[20]	valid_0's rmse: 0.520405
[21]	valid_0's rmse: 0.520429
[22]	valid_0's rmse: 0.520557
[23]	valid_0's rmse: 0.521221
[24]	valid_0's rmse: 0.52136
[25]	valid_0's rmse: 0.52148
[26]	valid_0's rmse: 0.521086
[27]	valid_0's rmse: 0.52136
[28]	valid_0's rmse: 0.521328
[29]	valid_0's rmse: 0.521451
Early stopping, best iteration is:
[9]	valid_0's rmse: 0.51874
The rmse of predicti

In [0]:
y_train_wb_ohe_50k_and_preds = gbm.predict(x_train_wb_ohe_50k_and_preds, num_iterations=gbm.best_iteration)

In [0]:
pickle.dump(y_train_wb_ohe_50k_and_preds, open('/content/gdrive/My Drive/MerCari/y_train_wb_ohe_50k_and_preds.p','wb'))
pickle.dump(y_test_wb_ohe_50k_and_preds, open('/content/gdrive/My Drive/MerCari/y_test_wb_ohe_50k_and_preds.p','wb'))

In [0]:
y_train_wb_ohe_50k_and_preds = pickle.load(open('/content/gdrive/My Drive/MerCari/y_train_wb_ohe_50k_and_preds.p','rb'))
y_test_wb_ohe_50k_and_preds = pickle.load(open('/content/gdrive/My Drive/MerCari/y_test_wb_ohe_50k_and_preds.p','rb'))

In [0]:
print(y_train_wb_ohe_ftrl.shape)
print(y_train_wb_ohe_fm_ftrl.shape)
print(y_train_wb_ohe_50k_and_preds.shape)

(1184672,)
(1184672,)
(1184672,)


In [0]:
y_train_wb_ohe_ftrl.reshape(y_train_wb_ohe_ftrl.shape[0],-1).shape

(1184672, 1)

In [0]:
y_train_ftrl_fm_frtl_lgbm = np.hstack((y_train_wb_ohe_ftrl.reshape(y_train_wb_ohe_ftrl.shape[0],-1),
                                      y_train_wb_ohe_fm_ftrl.reshape(y_train_wb_ohe_ftrl.shape[0],-1),
                                      y_train_wb_ohe_50k_and_preds.reshape(y_train_wb_ohe_ftrl.shape[0],-1)))

y_test_ftrl_fm_frtl_lgbm = np.hstack((y_test_wb_ohe_ftrl.reshape(y_test_wb_ohe_ftrl.shape[0],-1),
                                      y_test_wb_ohe_fm_ftrl.reshape(y_test_wb_ohe_ftrl.shape[0],-1),
                                      y_test_wb_ohe_50k_and_preds.reshape(y_test_wb_ohe_ftrl.shape[0],-1)))

In [0]:
y_train_fm_frtl_lgbm = np.hstack((y_train_wb_ohe_fm_ftrl.reshape(y_train_wb_ohe_ftrl.shape[0],-1),
                                  y_train_wb_ohe_50k_and_preds.reshape(y_train_wb_ohe_ftrl.shape[0],-1)))

y_test_fm_frtl_lgbm = np.hstack((y_test_wb_ohe_fm_ftrl.reshape(y_test_wb_ohe_ftrl.shape[0],-1),
                                      y_test_wb_ohe_50k_and_preds.reshape(y_test_wb_ohe_ftrl.shape[0],-1)))

In [0]:
model_fm_frtl_lgbm = Ridge(alpha=10, max_iter=100)
model_fm_frtl_lgbm.fit(y_train_fm_frtl_lgbm, y_train)
y_test_final = model_fm_frtl_lgbm.predict(y_test_fm_frtl_lgbm)
print('The rmse of prediction is:', mean_squared_error(y_test, y_test_final) ** 0.5)

The rmse of prediction is: 0.5034666634455569


In [0]:
model_frtrl_fm_frtl_lgbm = Ridge(alpha=7, max_iter=100)
model_frtrl_fm_frtl_lgbm.fit(y_train_ftrl_fm_frtl_lgbm, y_train)
y_test_final = model_frtrl_fm_frtl_lgbm.predict(y_test_ftrl_fm_frtl_lgbm)
print('The rmse of prediction is:', mean_squared_error(y_test, y_test_final) ** 0.5)

The rmse of prediction is: 0.5130937957030122


In [0]:
from sklearn.svm import SVR
#svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
#svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
 #              coef0=1)
#y_rbf = svr_rbf.fit(y_train_ftrl_fm_frtl_lgbm, y_train).predict(y_test_ftrl_fm_frtl_lgbm)
y_lin = svr_lin.fit(y_train_ftrl_fm_frtl_lgbm, y_train).predict(y_test_ftrl_fm_frtl_lgbm)
#y_poly = svr_poly.fit(y_train_ftrl_fm_frtl_lgbm, y_train).predict(y_test_ftrl_fm_frtl_lgbm)


#print('The rmse of prediction is:', mean_squared_error(y_test, y_rbf) ** 0.5)
print('The rmse of prediction is:', mean_squared_error(y_test, y_lin) ** 0.5)
#print('The rmse of prediction is:', mean_squared_error(y_test, y_poly) ** 0.5)

In [0]:
y_test_final = y_test_wb_ohe_ftrl * 0.1  + y_test_wb_ohe_fm_ftrl * 0.5 + y_test_wb_ohe_50k_and_preds * 0.4

print('The rmse of prediction is:', mean_squared_error(y_test, y_test_final) ** 0.5)

The rmse of prediction is: 0.5000186144312139


In [0]:
lgb_train = lgb.Dataset(y_train_ftrl_fm_frtl_lgbm, y_train)
lgb_eval = lgb.Dataset(y_test_ftrl_fm_frtl_lgbm, y_test, reference=lgb_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'RMSE',
    'max_depth' : 20,
    'num_leaves': 120,
    'learning_rate': 1,
    'n_jobs' : -1,
    'verbose': 0
}
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_eval,
                early_stopping_rounds=20
                )

y_test_final = gbm.predict(y_test_ftrl_fm_frtl_lgbm, num_iterations=gbm.best_iteration)

print('The rmse of prediction is:', mean_squared_error(y_test, y_test_final) ** 0.5)

[1]	valid_0's rmse: 0.517326
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 0.527273
[3]	valid_0's rmse: 0.534359
[4]	valid_0's rmse: 0.535027
[5]	valid_0's rmse: 0.537251
[6]	valid_0's rmse: 0.538184
[7]	valid_0's rmse: 0.539456
[8]	valid_0's rmse: 0.540836
[9]	valid_0's rmse: 0.541111
[10]	valid_0's rmse: 0.540832
[11]	valid_0's rmse: 0.54172
[12]	valid_0's rmse: 0.542518
[13]	valid_0's rmse: 0.542526
[14]	valid_0's rmse: 0.543404
[15]	valid_0's rmse: 0.543692
[16]	valid_0's rmse: 0.544959
[17]	valid_0's rmse: 0.544528
[18]	valid_0's rmse: 0.545257
[19]	valid_0's rmse: 0.545801
[20]	valid_0's rmse: 0.546413
[21]	valid_0's rmse: 0.546657
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.517326
The rmse of prediction is: 0.5173255487020154


In [0]:
lgb_train = lgb.Dataset(y_train_fm_frtl_lgbm, y_train)
lgb_eval = lgb.Dataset(y_test_fm_frtl_lgbm, y_test, reference=lgb_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'RMSE',
    'max_depth' : 20,
    'num_leaves': 120,
    'learning_rate': 1,
    'n_jobs' : -1,
    'verbose': 0
}
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_eval,
                early_stopping_rounds=20
                )

y_test_final = gbm.predict(y_test_fm_frtl_lgbm, num_iterations=gbm.best_iteration)

print('The rmse of prediction is:', mean_squared_error(y_test, y_test_final) ** 0.5)

[1]	valid_0's rmse: 0.515567
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 0.524671
[3]	valid_0's rmse: 0.525789
[4]	valid_0's rmse: 0.527827
[5]	valid_0's rmse: 0.53105
[6]	valid_0's rmse: 0.531469
[7]	valid_0's rmse: 0.531661
[8]	valid_0's rmse: 0.531587
[9]	valid_0's rmse: 0.53116
[10]	valid_0's rmse: 0.53135
[11]	valid_0's rmse: 0.5322
[12]	valid_0's rmse: 0.532759
[13]	valid_0's rmse: 0.532962
[14]	valid_0's rmse: 0.53217
[15]	valid_0's rmse: 0.532347
[16]	valid_0's rmse: 0.532718
[17]	valid_0's rmse: 0.532951
[18]	valid_0's rmse: 0.532648
[19]	valid_0's rmse: 0.533325
[20]	valid_0's rmse: 0.534367
[21]	valid_0's rmse: 0.534514
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.515567
The rmse of prediction is: 0.5155668238503941


In [0]:
from __future__ import print_function, division, absolute_import
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import column_or_1d
#from sklearn.preprocessing.label import _check_numpy_unicode_bug
import numpy as np
import pandas as pd
#from skutil.base import BaseSkutil
#from skutil.utils import validate_is_pd

__all__ = [
    'SafeLabelEncoder',
    'OneHotCategoricalEncoder'
]


def _get_unseen():
    """Basically just a static method
    instead of a class attribute to avoid
    someone accidentally changing it."""
    return 99999


class SafeLabelEncoder(LabelEncoder):
    """An extension of LabelEncoder that will
    not throw an exception for unseen data, but will
    instead return a default value of 99999

    Attributes
    ----------

    classes_ : the classes that are encoded
    """

    def transform(self, y):
        """Perform encoding if already fit.

        Parameters
        ----------

        y : array_like, shape=(n_samples,)
            The array to encode

        Returns
        -------

        e : array_like, shape=(n_samples,)
            The encoded array
        """
        check_is_fitted(self, 'classes_')
        y = column_or_1d(y, warn=True)

        classes = np.unique(y)
        #_check_numpy_unicode_bug(classes)

        # Check not too many:
        unseen = _get_unseen()
        if len(classes) >= unseen:
            raise ValueError('Too many factor levels in feature. Max is %i' % unseen)

        e = np.array([
                         np.searchsorted(self.classes_, x) if x in self.classes_ else unseen
                         for x in y
                         ])

        return e